In [ ]:
import pandas as pd
import glob
import re 

arquivos = sorted(glob.glob('../data/*.xlsx'))


# Função para extrair o nome do mês do arquivo.
def extrair_mes(nome_arquivo): 
    nome = nome_arquivo.split("\\")[-1].split("/")[-1]
    nome = nome.replace('.xlsx', '').lower()
    meses = {'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4,
             'maio': 5, 'junho': 6, 'julho': 7, 'agosto': 8,
             'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12}
    for mes, numero in meses.items():
        if mes in nome:
            return numero
    return None


# Lista para armazenar os DataFrames de cada arquivo.
dfs = []

for arquivo in arquivos:
    print (f'Lendo o arquivo: {arquivo}')
    mes = extrair_mes(arquivo)
    
    #Lê a planilha (assumindo que a tabela começa na linha 3.)
    df = pd.read_excel(arquivo, header=2)
    
    
    #Renomeias a primeira coluna
    df = df.rename(columns={df.columns[0]: 'UF'})


    #Adiciona a coluna 'Mês'
    df["Mês"] = mes
    
    #Mantém apenas as colunas relevantes 
    colunas = ["UF", "TOTAL", "AUTOMÓVEL", "MOTOCICLETA", "mes"]
    df = df[[c for c in colunas if c in df.columns]]
    
    
    dfs.append(df)
    
#Junta todos os meses
df_consolidado = pd.concat(dfs, ignore_index=True)

#Limpeza: remove linhas agregadas (Brasil, regiões)
df_consolidado = df_consolidado[~df_consolidado["UF"].isin(
    ["Brasil", "Norte", "Nordeste", "Sudeste", "Sul", "Centro-Oeste"]
)]
    
# Conversão de colunas numéricas (com tratamento de vazios e hifens)
for col in ["TOTAL", "AUTOMÓVEL", "MOTOCICLETA"]:
    df_consolidado[col] = (
        df_consolidado[col]
        .astype(str)
        .str.replace(r"[^0-9]", "", regex=True)  # mantém só números
        .replace("", None)                       # troca strings vazias por None
    )
    df_consolidado[col] = pd.to_numeric(df_consolidado[col], errors="coerce")  # converte seguro


    
df_consolidado.to_csv(
    "../results/frota_consolidada_2025.csv",
    index=False,
    sep=";",               # separador padrão do Excel PT-BR
    encoding="latin-1",    # evita os caracteres bugados
    decimal=","            # vírgula como separador decimal (opcional)
) 
    


print("✅ Consolidação finalizada!")
print("Arquivo salvo em: results/frota_consolidada_2025.csv")





Lendo: ../data\Abril.xlsx
Lendo: ../data\Agosto.xlsx
Lendo: ../data\Fevereiro.xlsx
Lendo: ../data\Janeiro.xlsx
Lendo: ../data\Julho.xlsx
Lendo: ../data\Junho.xlsx
Lendo: ../data\Maio.xlsx
Lendo: ../data\Marco.xlsx
Lendo: ../data\Setembro.xlsx
✅ Consolidação finalizada!
Arquivo salvo em: results/frota_consolidada_2025.csv
